In [1]:
import tensorflow as tf 
# import numpy as np 
print(tf.__version__)

print(f'Number of GPU\'s available:', len(tf.config.list_physical_devices('GPU')))

2.6.0
Number of GPU's available: 1


`TextVectorization` Layer will be used for word splitting and indexing. 

# Setup / Load data

In [3]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 80.2M    0  320k    0     0   232k      0  0:05:53  0:00:01  0:05:52  232k
  1 80.2M    1 1488k    0     0   628k      0  0:02:10  0:00:02  0:02:08  628k
  4 80.2M    4 3584k    0     0  1064k      0  0:01:17  0:00:03  0:01:14 1064k
  8 80.2M    8 7200k    0     0  1646k      0  0:00:49  0:00:04  0:00:45 1645k
 15 80.2M   15 12.7M    0     0  2431k      0  0:00:33  0:00:05  0:00:28 2672k
 25 80.2M   25 20.2M    0     0  3263k      0  0:00:25  0:00:06  0:00:19 4101k
 38 80.2M   38 30.8M    0     0  4285k      0  0:00:19  0:00:07  0:00:12 6017k
 52 80.2M   52 42.2M    0     0  5177k      0  0:00:15  0:00:08  0:00:07 7947k
 67 80.2M   67 54.2M    0     0  5934k      0  0:00

In [2]:
import os 
os.listdir('aclImdb')

['imdb.vocab', 'imdbEr.txt', 'README', 'test', 'train']

In [5]:
!cat aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

In [6]:
# usup is unsupervised dataset (do not need it for text classification)
!rm -r aclImdb/train/unsup

You can use `tf.keras.preprocessing.text_dataset_from_directory` to generate a labeled `tf.data.Dataset` object from a set of text files on disk filed into class-specific folders. 

Will generate 80% training 20% validation form the train folder. 

Validation set is uesful for tuning hyperparameters, such as model architecture, for which the test dataset should not be used. Before productionizing the model, it should be retrained using all available training data (without creating a validation set) so its performance is maximized. 

When using `validation_split` & `subset` make sure to pass random seed or `shuffle=False` so that the validation & training splits have no overlap. 

In [3]:
batch_size = 32 
validation_split = 0.2 
seed = 1337

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=validation_split, 
    subset='training', 
    seed=seed,
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=validation_split, 
    subset='validation', 
    seed=seed
)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size = batch_size 
)

print(f'Number of batches in raw_train_ds: {raw_train_ds.cardinality()}')
print(f'Number of batches in raw_val_ds: {raw_val_ds.cardinality()}')
print(f'Number of batches in raw_test_ds: {raw_test_ds.cardinality()}')

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


In [4]:
# using eager execution instead of Session/Graph context to see the raw data and 
# figure out how normalization will work 

for text_batch, label_batch in raw_train_ds.take(1): # take 1 batch 
    for i in range(3): # take the first 3 values in the batch (out of batch_size=32) 
        print(label_batch.numpy()[i], '=>', str(text_batch.numpy()[i]))
        

1 => b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receiv

# Prepare the Data 

The `<br />` needs to be removed. 

In [5]:
import string 
import re 

# normal text standardizer does not strip HTML tags from text. 
# custom standardization function needs to be created 
def custom_standardization(input_data): 
    lowercase = tf.strings.lower(input_data) 
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ') 
    return tf.strings.regex_replace(
        stripped_html, f'[{re.escape(string.punctuation)}]', ''
    )

In [6]:
from tensorflow.keras.layers import TextVectorization 


# model constants 
max_features = 20_000 
embedding_dim = 128 
sequence_length = 500 

# after standardization we can instantiate out text vectorization layer 
# using this layer to normalize, split, and map strings to integers thus setting out `output_mode` to `int`
# setting an explicit maximum sequence length since CNN layers will not support ragged sequences 
vectorize_layer = TextVectorization(
    standardize=custom_standardization, 
    max_tokens=max_features, 
    output_mode='int', 
    output_sequence_length=sequence_length
)

# text only dataset 
text_ds = raw_train_ds.map(lambda x,y:x)
vectorize_layer.adapt(text_ds)

## Two options to vectorize the data 

__Option 1: Make it part of the model__, so as to obtain a model that processes raw strings

```python 
text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input) 
x = layers.Embedding(max_features+1, embedding_dim)(x)
...
```

__Option 2: Apply it to the text dataset__, to obtain a dataset of word indices, then feed it into a model that expects integer sequences as inputs. 

An important difference between the two is that option 2 enables you to do _async CPU processing and buffering_ of your data when training on GPU. So if you're training the model on the GPU, you probably want to go with this options to get the best performance. 

If we were to export our model to production, we'd ship a model that accepts raw strings as input. 

In [7]:
def vectorize_text(text, label): 
    text = tf.expand_dims(text, -1) 
    return vectorize_layer(text), label 

# Vectorize the data 
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU 
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10) 
test_ds = test_ds.cache().prefetch(buffer_size=10)

# Build the Model 

Simple 1D convnet with an `Embedding` layer 

In [10]:
from tensorflow.keras import layers 

inputs = tf.keras.Input(shape=(None,), dtype='int64')

# map vocab indicies into a space of dimensionality 
x = layers.Embedding(max_features, embedding_dim)(inputs) 
x = layers.Dropout(0.5)(x) 

# Conv1D + global max pooling (1D convnet becuase it is text data ) 
x = layers.Conv1D(128, 7, padding='valid', activation='relu', strides=3)(x)
x = layers.Conv1D(128, 7, padding='valid', activation='relu', strides=3)(x)
x = layers.GlobalAveragePooling1D()(x) 

# vanilla hidden layer 
x = layers.Dense(128, activation='relu')(x) 
x = layers.Dense(0.5)(x) 

# project onto a single unit output layer and squash it with a sigmoid ([0,1] range i.e. probability of it being positive or negative)
predictions = layers.Dense(1, activation='sigmoid', name='predictions')(x) 
model = tf.keras.Model(inputs, predictions, name='text_classification') 
print(model.summary())


Model: "text_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 128)         114816    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 128)         114816    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)       

In [11]:
# binary crossentropy and adam optimizer 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model Training 

In [ ]:
epochs = 3 

model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Error: Kernel is dead

In [12]:
model.evaluate(test_ds)

782/782 [==============================] - 39s 46ms/step - loss: 0.6931 - accuracy: 0.5000


[0.6931474804878235, 0.5]